In [1]:
import pandas as pd
import sqlite3

In [30]:
sql_conn = sqlite3.connect('../data/checking-logs.sqlite')

In [31]:
query = "PRAGMA table_info(test);"
pd.io.sql.read_sql(query, sql_conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


In [32]:
query = "SELECT * FROM test LIMIT 10"
pd.io.sql.read_sql(query, sql_conn)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [33]:
query = """
SELECT
    t.uid,
    MIN(CAST((JulianDay(t.first_commit_ts) - (JulianDay(d.deadlines, 'unixepoch')))* 24 AS INTEGER)) AS diff
FROM test t
JOIN deadlines d
    ON t.labname = d.labs
WHERE
    labname <> 'project1'
"""
df_min = pd.io.sql.read_sql(query, sql_conn)
df_min

,uid,diff
0,user_30,-202


In [34]:
query = """
SELECT
    t.uid,
    MAX(CAST((JulianDay(t.first_commit_ts) - (JulianDay(d.deadlines, 'unixepoch'))) * 24 AS INTEGER)) AS diff
FROM test t
JOIN deadlines d
    ON t.labname = d.labs
WHERE
    labname <> 'project1'
"""
df_max = pd.io.sql.read_sql(query, sql_conn)
df_max

,uid,diff
0,user_25,-2


In [35]:
query = """
SELECT
    AVG(CAST((JulianDay(t.first_commit_ts) - (JulianDay(d.deadlines, 'unixepoch')))* 24 AS INTEGER)) AS diff
FROM test t
JOIN deadlines d
    ON t.labname = d.labs
WHERE
    labname <> 'project1'
"""
df_avg = pd.io.sql.read_sql(query, sql_conn)
df_avg

,diff
0,-89.125


In [36]:
query = 'SELECT  * FROM pageviews WHERE uid LIKE "user%"'
pd.io.sql.read_sql(query, sql_conn)

,index,uid,datetime
0,12,user_30,2020-04-17 22:46:26.785035
1,14,user_14,2020-04-18 10:53:52.623447
2,15,user_17,2020-04-18 10:56:55.833899
3,16,user_14,2020-04-18 10:57:37.331258
4,18,user_17,2020-04-18 12:05:48.200144
...,...,...,...
982,1073,user_28,2020-05-21 18:45:20.441142
983,1074,user_19,2020-05-21 23:03:06.457819
984,1075,user_25,2020-05-21 23:23:49.995349
985,1078,user_5,2020-05-22 11:30:18.368990


In [37]:
query = """
WITH tmp AS (SELECT
    test.uid,
    AVG(CAST((JulianDay(test.first_commit_ts) - (JulianDay(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER)) AS avg_dif
FROM test
JOIN deadlines
    ON deadlines.labs = test.labname
GROUP BY test.uid)
SELECT tmp.uid AS uid,
    tmp.avg_dif,
    COUNT(pageviews.datetime) AS pageviews
FROM tmp
INNER JOIN pageviews
    ON tmp.uid = pageviews.uid
GROUP BY tmp.uid
"""
views_df = pd.io.sql.read_sql(query, sql_conn)
views_df.corr()

,avg_dif,pageviews
avg_dif,1.000000,-0.069302
pageviews,-0.069302,1.000000


In [25]:
sql_conn.close()